<h1 style='text-align: center; front-size: 50px;'>Instacart Customer Behavior Analysis</h1>

# Itroduction:
 In this project, we will work with data from "instacart". Instacart is a grocery delivery platform where customers
 can place a grocery order and have it delivered to them, similar to how Uber Eats and Door Dash work. It was montioned
 that the dataset provided has been modified from the original for a better and faster calculations.
 Our mission is to clean up the data and prepare a report that gives insight into the shopping habits
 of Instacart customers.
 There are five tables in the dataset, and we’ll need to use all of them to do our data preprocessing. They are stored in:
    /datasets/instacart_orders.csv,
    /datasets/products.csv,
    /datasets/aisles.csv,
    /datasets/departments.csv,
    /datasets/order_products.csv
 During our data preprocessing we will:
    Display the dataset following to the standardized format,
    Verify and fix data types,
    Identify and fill in missing values,
    Identify and remove duplicate values,
    Create plots that communicate clear and concise understanding of the data.


In [ ]:
import pandas as pd
orders = pd.read_csv ('/datasets/instacart_orders.csv', sep = ';')
orders.head()


In [ ]:
import pandas as pd
products = pd.read_csv('/datasets/products.csv', sep = ';')
products.head()


In [ ]:
import pandas as pd
aisles = pd.read_csv('/datasets/aisles.csv', sep = ';')
aisles.head()


In [ ]:
import pandas as pd
departments = pd.read_csv('/datasets/departments.csv', sep = ';')
departments.head()


In [ ]:
import pandas as pd
order_products = pd.read_csv('/datasets/order_products.csv', sep = ';')
order_products.head()

## Find and remove duplicate values:

### `orders` data frame

In [ ]:
# Check for duplicated orders
orders.duplicated().sum()


In [ ]:
# Check for all orders placed Wednesday at 2:00 AM
wednesday_orders = orders[(orders['order_dow'] == 3) & (orders['order_hour_of_day'] == 2)]
wednesday_orders

In [ ]:
# Remove duplicate orders
orders.drop_duplicates(inplace=True)

In [ ]:
# Double check for duplicate rows
orders.duplicated().sum()

In [ ]:
# Double check for duplicate order IDs only
orders.duplicated(subset='order_id').sum()

We identified 15 missing values. The missing values are not critical, so we droped all rows with missing values.

### `products` data frame

In [ ]:
# Check for fully duplicate rows
products.duplicated().sum()


In [ ]:
# Check for just duplicate product IDs
products.duplicated(subset='product_id').sum()

In [ ]:
# Check for just duplicate product names (convert names to lowercase to compare better)
products['product_name'].str.lower().duplicated().sum()


In [ ]:
# Check for duplicate product names that aren't missing
products['product_name'].dropna().str.lower().duplicated().sum()

Converting product names to lowercase showed 1361 duplicates on the missing values and 104 on the non missing values. They should be adressed to ensure data accuracy

### `departments` data frame

In [ ]:
departments.duplicated().sum()


For departments dataset we noticed that there is no duplicated values

### `aisles` data frame

In [ ]:
aisles.duplicated().sum()

For aisles dataset we noticed that there is no duplicated values

### `order_products` data frame

In [ ]:
# Check for fullly duplicate rows
order_products.duplicated().sum()

In [ ]:
# Double check for any other tricky duplicates
order_products.duplicated(subset =['order_id', 'product_id']).sum()

For order_products dataset we noticed that there is no duplicated values

## Find and remove missing values


### `products` data frame

In [ ]:
products.isna().sum()

In [ ]:
# Are all of the missing product names associated with aisle ID 100?
products[(products['product_name'].isna()) & (products['aisle_id'] == 100)]

In [ ]:
# Are all of the missing product names associated with department ID 21?
products[(products['product_name'].isna()) & (products['department_id'] == 21)]

In [ ]:
# What is this ailse and department?
filtered_department = departments[departments['department_id'] == 21]
filtered_department

In [ ]:
# Added by reviewer
filtered_aisles = aisles[aisles['aisle_id'] == 100]
filtered_aisles

In [ ]:
# Fill missing product names with 'Unknown'
products['product_name'].fillna('Unknown', inplace=True)

### `orders` data frame

In [ ]:
# Are there any missing values where it's not a customer's first order?
not_first_orders = orders[orders['order_number'] > 1]

missing_values = not_first_orders['days_since_prior_order'].isnull().sum()

missing_values

No missing values were found in the days_since_prior_order column for orders beyond the customer's first order.

### `order_products` data frame

In [ ]:
# What are the min and max values in this column?
order_products['add_to_cart_order'].min(), order_products['add_to_cart_order'].max()


In [ ]:
# Save all order IDs with at least one missing value in 'add_to_cart_order'
missing_order_ids = order_products[order_products['add_to_cart_order'].isna()]['order_id'].unique()
missing_order_ids

In [ ]:
# Do all orders with missing values have more than 64 products?
order_counts = order_products[order_products['order_id'].isin(missing_order_ids)].groupby('order_id').size()
check = (order_counts > 64).all()
check

In [ ]:
# Replace missing values with 999 and convert column to integer type
order_products['add_to_cart_order'].fillna(999).astype('int')


After our analysis we can say that all orders with missing values have more than 64 products.

In [ ]:
# Check for 'order_hour_of_day':
if (orders['order_hour_of_day'].min() >= 0) and (orders['order_hour_of_day'].max() <= 23):
    print("'order_hour_of_day' values are sensible.")
else:
    print("'order_hour_of_day' values are not sensible.")

# Check for 'order_dow':
if (orders['order_dow'].min() >= 0) and (orders['order_dow'].max() <= 6):
    print("'order_hour_of_day' values are sensible.")
else:
    print("'order_hour_of_day' values are not sensible.")

### What time of day do people shop for groceries?

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,2))
orders['order_hour_of_day'].value_counts().sort_index().plot(kind='bar')
plt.title('Number of Orders by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Orders')

plt.show()

After filtering the orders dataframe based on hour of the day, we noticed that the number of orders is at its peak between 9 a.m and 5 p.m.

### What day of the week do people shop for groceries?

In [ ]:
plt.figure(figsize=(6,2))
orders['order_dow'].value_counts().sort_index().plot(kind='bar')
plt.title('Number of Orders by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Orders')

plt.show() 

According to the plot, the number of orders is steady during all days of the week, but we noticed that their groceries shopping are in there peak on the first 2 days of the week.

### How long do people wait until placing another order?

In [ ]:
plt.figure(figsize=(6,2))
orders['days_since_prior_order'].plot(kind='hist', bins=15)
plt.title('Duration of Days Since Prior Order')
plt.xlabel('Days Since Prior Order')
plt.ylabel('Duration')

plt.show()  

The plot is showing that the majority of people wait from a day to 7 days before placing their second order. We also noticed that some people tend to wait 30 days till their second order.

### Is there a difference in `'order_hour_of_day'` distributions on Wednesdays and Saturdays?

In [ ]:
wednesday_orders = orders[orders['order_dow'] == 3]
saturday_orders = orders[orders['order_dow'] == 6]

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(wednesday_orders['order_hour_of_day'], bins=24, alpha=0.5, label='Wednesday')
plt.hist(saturday_orders['order_hour_of_day'], bins=24, alpha=0.5, label='Saturday')
plt.xlabel('Hour of  The Day')
plt.ylabel('Number of Orders')
plt.title('Orders by Hour on Wednesday vs. Saturday')
plt.xticks(range(0, 25))
plt.legend()

plt.show()

From the plot above, we can say that the number of orders on Wednesday and Saturday are similer, with a slight increase on Saturday between 10 a.m and 3 p.m.

### What's the distribution for the number of orders per customer?

In [ ]:
customer_order_counts = orders['user_id'].value_counts()
customer_order_counts

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(customer_order_counts, bins=30, alpha=0.7)
plt.title('Number of Orders per Customer')
plt.xlabel('Number of Orders')
plt.ylabel('Number of Customers')

plt.show()

Based on the plot, the data indicates that most customers place at least 4 orders, but there is a noticeable drop in the number of customers once they reach the 5 order mark.

### What are the top 20 popular products?

In [ ]:
merged_datasets = order_products.merge(products, on="product_id")
top_products = (merged_datasets.groupby(["product_id", "product_name"]).size().sort_values(ascending=False))

In [ ]:
top_products.head(20)

After filtering the ordered products data, we can say that perishables, such as fruits and vegetables, are the dominant products.

### How many items do people typically buy in one order?

In [ ]:
items_per_order = order_products.groupby('order_id').size().sort_values(ascending=False)
items_per_order

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.hist(items_per_order, bins=30, alpha=0.7)
plt.title('Items per Order')
plt.xlabel('Number of Items')
plt.ylabel('Orders')

plt.show()

From the plot, we can say that the higher the number of items bought the number of people drops. Typically, the number of items bought can range between 127 and 1 item per order.

### What are the top 20 items that are reordered most frequently?

In [ ]:
top_20_reorder = order_products[order_products['reordered'] == 1]['product_id'].value_counts().head(20).reset_index()
top_20_reorder.columns = ['product_id', 'count'] 
top_20_reordered = top_20_reorder.merge(products[['product_id', 'product_name']], on='product_id')
top_20_reordered

After looking into the ordered products data, we noticed that the top 20 reorder products are mostly perishables such as vegetables and fruits.

# General Conclusion:

Based on the datasets provided: 

  - We noticed that the number of orders is at its peak between 9 a.m and 5 p.m.
  
  - The number of orders is steady during all days of the week, but we noticed that people tend to do more groceries on the first 2 days of the week.
  
  - The majority of people wait from a day to 7 days before placing their second order. We also noticed that some people tend to wait 30 days till their second order.
  
  - We can say that the number of orders on Wednesday and Saturday are similer, with a slight increase on Saturday between 10 a.m and 3 p.m.
  
  - Most customers place at least 4 orders, but there is a noticeable drop in the number of customers once they reach the 5 order mark.
  
  - We can say that the higher the number of items bought the number of people drops.
  
  - We noticed that the top 20 reorder products are mostly perishables such as vegetables and fruits.
